In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore') 
from pylab import rcParams
rcParams["figure.figsize"]=30,16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/v11/v11_3.xlsx",sheet_name = 'ren_ln_orig')
data.head()

,OpportunityID_ren,accountnumber,OpportunityID,lnlexidsele,inputcheckbusname,inputcheckbusaltname,inputcheckbusaddr,inputcheckbuscity,inputcheckbusstate,inputcheckbuszip,...,model1rc5,model1rc6,model2name,model2score,model2rc1,model2rc2,model2rc3,model2rc4,model2rc5,model2rc6
0,0062M00000gYqZQQA0,LCFLN202208126653,0062M00000gYqZQQA0,1.069630e+08,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,SLBO1702_0_2,560.0,B078,B026,B063,B074,NaN,NaN
1,0062M00000gYSpGQAW,LCFLN202208126658,0062M00000gYSpGQAW,5.004454e+07,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,SLBO1702_0_2,723.0,B069,B063,B031,B066,NaN,NaN
2,0062M00000gYsQcQAK,LCFLN202208126659,0062M00000gYsQcQAK,9.550183e+10,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,SLBO1702_0_2,702.0,B034,B052,B036,B037,NaN,NaN
3,0062M00000n0BEfQAM,LCFLN202208130965,0062M00000n0BEfQAM,1.619373e+08,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,SLBO1702_0_2,562.0,B078,B026,B074,B063,NaN,NaN
4,0062M00000gYTaCQAW,LCFLN202208126663,0062M00000gYTaCQAW,2.354679e+07,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,SLBO1702_0_2,573.0,B063,B026,B078,B074,NaN,NaN


In [ ]:
data.shape

(3382, 392)

In [ ]:
'WriteOff_YN' in data.columns

False

In [ ]:
# getting write_offs from another sheet i.e. renewal_orig
wo = pd.read_excel("/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/v11/v11_3.xlsx",sheet_name = 'Ren_orig')

In [ ]:
'WriteOff_YN' in wo.columns

True

In [ ]:
wo.shape

(3539, 55)

In [ ]:
wo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3539 entries, 0 to 3538
Data columns (total 55 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   OpportunityID                              3539 non-null   object        
 1   CreatedDate                                3539 non-null   datetime64[ns]
 2   FundedDate                                 3539 non-null   datetime64[ns]
 3   OpportunityName                            3539 non-null   object        
 4   Priordeal_OppID                            3539 non-null   object        
 5   Renewal_PreviousCollectionStatus           3539 non-null   object        
 6   Renewal_PreviousPace                       3525 non-null   float64       
 7   Renewal_PreviousPaidPercentageOfRTR        3525 non-null   float64       
 8   Renewal_PreviousCreditScore                3495 non-null   float64       
 9   Renewal_CountPrevio

In [ ]:
# Considering Opportunity_ID and WriteOff_YN from Renewal_Exisiting_Attributes
wo = wo[['OpportunityID','WriteOff_YN']]

In [ ]:
wo

,OpportunityID,WriteOff_YN
0,0062M00000mTGVnQAO,1.0
1,0062M00000o96yhQAA,0.0
2,0062M00000mfz6iQAA,0.0
3,0062M00000lVi6gQAC,0.0
4,0062M00000mg31dQAA,0.0
...,...,...
3534,0062M00000oY7fdQAC,1.0
3535,0062M00000bmKPLQA2,1.0
3536,0062M00000bmhkAQAQ,1.0
3537,0062M00000lWtjxQAC,0.0


In [ ]:
d = pd.merge(data, wo, on="OpportunityID")

In [ ]:
d

,OpportunityID_ren,accountnumber,OpportunityID,lnlexidsele,inputcheckbusname,inputcheckbusaltname,inputcheckbusaddr,inputcheckbuscity,inputcheckbusstate,inputcheckbuszip,...,model1rc6,model2name,model2score,model2rc1,model2rc2,model2rc3,model2rc4,model2rc5,model2rc6,WriteOff_YN
0,0062M00000gYqZQQA0,LCFLN202208126653,0062M00000gYqZQQA0,1.069630e+08,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,560.0,B078,B026,B063,B074,NaN,NaN,0.0
1,0062M00000gYSpGQAW,LCFLN202208126658,0062M00000gYSpGQAW,5.004454e+07,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,723.0,B069,B063,B031,B066,NaN,NaN,0.0
2,0062M00000gYsQcQAK,LCFLN202208126659,0062M00000gYsQcQAK,9.550183e+10,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,702.0,B034,B052,B036,B037,NaN,NaN,0.0
3,0062M00000n0BEfQAM,LCFLN202208130965,0062M00000n0BEfQAM,1.619373e+08,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,562.0,B078,B026,B074,B063,NaN,NaN,0.0
4,0062M00000gYTaCQAW,LCFLN202208126663,0062M00000gYTaCQAW,2.354679e+07,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,573.0,B063,B026,B078,B074,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3377,0062M00000lTxv8QAC,LCFLN202208129165,0062M00000lTxv8QAC,1.197487e+11,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,659.0,B026,B069,B066,B070,NaN,NaN,0.0
3378,0062M00000lTxyLQAS,LCFLN202208129166,0062M00000lTxyLQAS,1.342359e+11,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,688.0,B034,B069,B037,B036,NaN,NaN,0.0
3379,0062M00000jzsJkQAI,LCFLN202208128064,0062M00000jzsJkQAI,6.488768e+10,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,711.0,B017,B066,B052,B036,NaN,NaN,1.0
3380,0062M00000l01oMQAQ,LCFLN202208128630,0062M00000l01oMQAQ,1.368324e+11,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,SLBO1702_0_2,222.0,B068,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
d.columns

Index(['OpportunityID_ren', 'accountnumber', 'OpportunityID', 'lnlexidsele',
       'inputcheckbusname', 'inputcheckbusaltname', 'inputcheckbusaddr',
       'inputcheckbuscity', 'inputcheckbusstate', 'inputcheckbuszip',
       ...
       'model1rc6', 'model2name', 'model2score', 'model2rc1', 'model2rc2',
       'model2rc3', 'model2rc4', 'model2rc5', 'model2rc6', 'WriteOff_YN'],
      dtype='object', length=393)

In [ ]:
data = d.copy()

In [ ]:
data.shape

(3382, 393)

In [ ]:
for i in data.columns:
  print(i)

OpportunityID_ren
accountnumber
OpportunityID
lnlexidsele
inputcheckbusname
inputcheckbusaltname
inputcheckbusaddr
inputcheckbuscity
inputcheckbusstate
inputcheckbuszip
inputcheckbusTIN
inputcheckbusphone
inputcheckbusfax
inputcheckbusnaics
inputcheckbussic
inputcheckbusage
inputcheckauthrepfirstname
inputcheckauthrepmiddlename
inputcheckauthreplastname
inputcheckauthrepaddr
inputcheckauthrepcity
inputcheckauthrepstate
inputcheckauthrepzip
inputcheckauthrepssn
inputcheckauthrepphone
inputcheckauthrepage
inputcheckauthrepdob
inputcheckauthrepdl
inputcheckauthrepdlstate
inputcheckauthrep2firstname
inputcheckauthrep2middlename
inputcheckauthrep2lastname
inputcheckauthrep2addr
inputcheckauthrep2city
inputcheckauthrep2state
inputcheckauthrep2zip
inputcheckauthrep2ssn
inputcheckauthrep2phone
inputcheckauthrep2age
inputcheckauthrep2dob
inputcheckauthrep2dl
inputcheckauthrep2dlstate
inputcheckauthrep3firstname
inputcheckauthrep3middlename
inputcheckauthrep3lastname
inputcheckauthrep3addr
input

In [ ]:
bankruptcy = ['bankruptcycount', 'bankruptcycount12m',
              'bankruptcycount24m', 'bankruptcycount84m', 'bankruptcychapter', 'bankruptcytimenewest',
              'OpportunityID', 'WriteOff_YN']
len(bankruptcy)

8

In [ ]:
df = data[bankruptcy]

In [ ]:
df.columns

Index(['bankruptcycount', 'bankruptcycount12m', 'bankruptcycount24m',
       'bankruptcycount84m', 'bankruptcychapter', 'bankruptcytimenewest',
       'OpportunityID', 'WriteOff_YN'],
      dtype='object')

In [ ]:
df.shape

(3382, 8)

In [ ]:
df.describe()

,bankruptcycount,bankruptcycount12m,bankruptcycount24m,bankruptcycount84m,bankruptcychapter,bankruptcytimenewest,WriteOff_YN
count,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000
mean,-0.138971,-0.149320,-0.146067,-0.139562,-0.043170,0.298936,0.264341
std,0.377057,0.358937,0.364760,0.376053,1.100092,5.037358,0.441047
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,11.000000,104.000000,1.000000


In [ ]:
round(df.corr()['WriteOff_YN'].sort_values(),4)

bankruptcychapter      -0.0228
bankruptcytimenewest   -0.0223
bankruptcycount        -0.0209
bankruptcycount84m     -0.0200
bankruptcycount24m     -0.0191
bankruptcycount12m     -0.0159
WriteOff_YN             1.0000
Name: WriteOff_YN, dtype: float64

In [ ]:
for i in df.columns:
  print('\033[1m'+'attribute_name :', i+'\033[0m')
  print(df[i].value_counts())

attribute_name : bankruptcycount
 0.0    2836
-1.0     508
 1.0      38
Name: bankruptcycount, dtype: int64
attribute_name : bankruptcycount12m
 0.0    2871
-1.0     508
 1.0       3
Name: bankruptcycount12m, dtype: int64
attribute_name : bankruptcycount24m
 0.0    2860
-1.0     508
 1.0      14
Name: bankruptcycount24m, dtype: int64
attribute_name : bankruptcycount84m
 0.0    2838
-1.0     508
 1.0      36
Name: bankruptcycount84m, dtype: int64
attribute_name : bankruptcychapter
 0.0     2836
-1.0      508
 11.0      24
 7.0       14
Name: bankruptcychapter, dtype: int64
attribute_name : bankruptcytimenewest
 0.0      2836
-1.0       508
 16.0        2
 15.0        2
 28.0        2
 75.0        2
 20.0        1
 47.0        1
 84.0        1
 21.0        1
 54.0        1
 61.0        1
 6.0         1
 50.0        1
 27.0        1
 31.0        1
 4.0         1
 55.0        1
 58.0        1
 41.0        1
 53.0        1
 13.0        1
 43.0        1
 8.0         1
 39.0        1
 45.0   

In [ ]:
df.isna().sum()

bankruptcycount         0
bankruptcycount12m      0
bankruptcycount24m      0
bankruptcycount84m      0
bankruptcychapter       0
bankruptcytimenewest    0
OpportunityID           0
WriteOff_YN             0
dtype: int64

In [ ]:
df.shape

(3382, 8)

In [ ]:
df.drop(columns =["bankruptcychapter"],axis = 1, inplace = True)

In [ ]:
df.shape

(3382, 7)

In [ ]:
df['bankruptcytimenewest_value'] = df["bankruptcytimenewest"].apply(lambda x:0 if x < 0 else x)
df['bankruptcytimenewest_flag'] = df["bankruptcytimenewest"].apply(lambda x:1 if x < 0 else 0)

In [ ]:
df[df["OpportunityID"] == '0062M00000itRUCQA2']["bankruptcytimenewest"]

10   -1.0
Name: bankruptcytimenewest, dtype: float64

In [ ]:
df[df["OpportunityID"] == '0062M00000noBRbQAM'][['bankruptcytimenewest', 'OpportunityID']]

,bankruptcytimenewest,OpportunityID
21,-1.0,0062M00000noBRbQAM


In [ ]:
df.drop(columns = ["bankruptcytimenewest"], axis = 1, inplace=True)

In [ ]:
df.columns

Index(['bankruptcycount', 'bankruptcycount12m', 'bankruptcycount24m',
       'bankruptcycount84m', 'OpportunityID', 'WriteOff_YN',
       'bankruptcytimenewest_value', 'bankruptcytimenewest_flag'],
      dtype='object')

In [ ]:
def bankruptcy_FE_categorical(df):
  df[['bankruptcycount_f1', 'bankruptcycount_f2', 'bankruptcycount_f3',
      'bankruptcycount12m_f1',"bankruptcycount12m_f2", "bankruptcycount12m_f3",
      'bankruptcycount24m_f1', 'bankruptcycount24m_f2', 'bankruptcycount24m_f3',
      'bankruptcycount84m_f1', 'bankruptcycount84m_f2',  'bankruptcycount84m_f3']] = 0
  
  for i in ['bankruptcycount', 'bankruptcycount12m', 'bankruptcycount24m',
            'bankruptcycount84m']:
        df[i+"_f1"] = df[i].apply(lambda x: 1 if x== -1 else 0)
        df[i+"_f2"] = df[i].apply(lambda x: 1 if x == 0 else 0)
        df[i+"_f3"] = df[i].apply(lambda x: 1 if x == 1 else 0)
        df.drop(i, axis = 'columns', inplace = True)

In [ ]:
# function call 
bankruptcy_FE_categorical(df)

In [ ]:
df.shape

(3382, 16)

In [ ]:
# basic function check
df.columns

Index(['OpportunityID', 'WriteOff_YN', 'bankruptcytimenewest_value',
       'bankruptcytimenewest_flag', 'bankruptcycount_f1', 'bankruptcycount_f2',
       'bankruptcycount_f3', 'bankruptcycount12m_f1', 'bankruptcycount12m_f2',
       'bankruptcycount12m_f3', 'bankruptcycount24m_f1',
       'bankruptcycount24m_f2', 'bankruptcycount24m_f3',
       'bankruptcycount84m_f1', 'bankruptcycount84m_f2',
       'bankruptcycount84m_f3'],
      dtype='object')

In [ ]:
round(df.corr()['WriteOff_YN'].sort_values(),4)

bankruptcytimenewest_value   -0.0211
bankruptcycount12m_f2        -0.0204
bankruptcycount24m_f2        -0.0167
bankruptcycount84m_f2        -0.0150
bankruptcycount_f2           -0.0140
bankruptcycount_f3           -0.0130
bankruptcycount84m_f3        -0.0099
bankruptcycount24m_f3        -0.0073
bankruptcytimenewest_flag     0.0182
bankruptcycount_f1            0.0182
bankruptcycount12m_f1         0.0182
bankruptcycount24m_f1         0.0182
bankruptcycount84m_f1         0.0182
bankruptcycount12m_f3         0.0272
WriteOff_YN                   1.0000
Name: WriteOff_YN, dtype: float64

In [ ]:
df.to_excel("/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/v11/bankruptcy_attributes.xlsx", index = False)

# Firmographic Attributes

In [ ]:
firmographic_attributes = ['firmnonprofit', 'firmirsretirementplan', 'firmageobserved', 'firmemployeecount',
                          'firmemployeerangecount', 'firmownershiptype', 'firmreportedsales', 'firmreportedsalesrange',
                          'firmreportedearnings', 'firmparentcompanyind', 'OpportunityID', 'WriteOff_YN']
len(firmographic_attributes)

12

In [ ]:
df = data[firmographic_attributes]
df.head(3)

,firmnonprofit,firmirsretirementplan,firmageobserved,firmemployeecount,firmemployeerangecount,firmownershiptype,firmreportedsales,firmreportedsalesrange,firmreportedearnings,firmparentcompanyind,OpportunityID,WriteOff_YN
0,0.0,0.0,29.0,12.0,40.0,0.0,1000000.0,70.0,0.0,1.0,0062M00000gYqZQQA0,0.0
1,0.0,0.0,74.0,44.0,50.0,0.0,3938000.0,80.0,-1.0,1.0,0062M00000gYSpGQAW,0.0
2,0.0,0.0,4.0,1.0,10.0,0.0,-1.0,-1.0,-1.0,1.0,0062M00000gYsQcQAK,0.0


In [ ]:
df.columns

Index(['firmnonprofit', 'firmirsretirementplan', 'firmageobserved',
       'firmemployeecount', 'firmemployeerangecount', 'firmownershiptype',
       'firmreportedsales', 'firmreportedsalesrange', 'firmreportedearnings',
       'firmparentcompanyind', 'OpportunityID', 'WriteOff_YN'],
      dtype='object')

In [ ]:
df.shape

(3382, 12)

In [ ]:
df.describe()

,firmnonprofit,firmirsretirementplan,firmageobserved,firmemployeecount,firmemployeerangecount,firmownershiptype,firmreportedsales,firmreportedsalesrange,firmreportedearnings,firmparentcompanyind,WriteOff_YN
count,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3.382000e+03,3382.000000,3382.000000,3382.000000,3382.000000
mean,-0.141336,-0.140154,6.948847,2.582200,11.178593,-0.150207,3.323386e+05,18.900059,304.185689,0.699586,0.264341
std,0.373018,0.375045,10.612479,6.955443,15.169516,0.357327,1.085638e+06,26.771284,10400.021782,0.714654,0.441047
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,0.000000
25%,0.000000,0.000000,1.000000,-1.000000,-1.000000,0.000000,-1.000000e+00,-1.000000,-1.000000,1.000000,0.000000
50%,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,-1.000000e+00,-1.000000,-1.000000,1.000000,0.000000
75%,0.000000,0.000000,9.000000,3.000000,20.000000,0.000000,2.080000e+05,40.000000,-1.000000,1.000000,1.000000
max,1.000000,1.000000,110.000000,68.000000,60.000000,0.000000,2.299900e+07,100.000000,499510.000000,1.000000,1.000000


In [ ]:
round(df.corr()['WriteOff_YN'].sort_values(),4)

firmageobserved          -0.1026
firmemployeerangecount   -0.0811
firmreportedsalesrange   -0.0708
firmemployeecount        -0.0381
firmirsretirementplan    -0.0263
firmnonprofit            -0.0191
firmparentcompanyind     -0.0182
firmownershiptype        -0.0182
firmreportedsales        -0.0142
firmreportedearnings      0.0101
WriteOff_YN               1.0000
Name: WriteOff_YN, dtype: float64

In [ ]:
for i in df.columns:
  print('\033[1m'+'attribute_name :', i+'\033[0m')
  print(df[i].value_counts())

attribute_name : firmnonprofit
 0.0    2844
-1.0     508
 1.0      30
Name: firmnonprofit, dtype: int64
attribute_name : firmirsretirementplan
 0.0    2840
-1.0     508
 1.0      34
Name: firmirsretirementplan, dtype: int64
attribute_name : firmageobserved
-1.0      517
 2.0      512
 1.0      398
 3.0      321
 4.0      228
         ... 
 51.0       1
 103.0      1
 74.0       1
 49.0       1
 85.0       1
Name: firmageobserved, Length: 62, dtype: int64
attribute_name : firmemployeecount
-1.0     1634
 2.0      380
 1.0      338
 0.0      150
 3.0      147
 4.0      102
 5.0       85
 6.0       66
 9.0       58
 7.0       54
 10.0      50
 13.0      39
 8.0       36
 11.0      29
 12.0      24
 18.0      23
 14.0      17
 15.0      16
 32.0      13
 23.0      12
 25.0      12
 21.0      11
 17.0      10
 19.0       7
 16.0       7
 30.0       6
 22.0       6
 50.0       5
 33.0       4
 40.0       4
 26.0       4
 31.0       3
 27.0       3
 41.0       3
 20.0       3
 24.0       3
 4

In [ ]:
df.isna().sum()

firmnonprofit             0
firmirsretirementplan     0
firmageobserved           0
firmemployeecount         0
firmemployeerangecount    0
firmownershiptype         0
firmreportedsales         0
firmreportedsalesrange    0
firmreportedearnings      0
firmparentcompanyind      0
OpportunityID             0
WriteOff_YN               0
dtype: int64

In [ ]:
df.shape

(3382, 12)

In [ ]:
# There are 3255 missing values in this feature
df.drop(['firmreportedearnings', 'firmreportedsales', 'firmemployeecount'], axis = 1, inplace = True)

In [ ]:
df.shape

(3382, 9)

In [ ]:
def firmographic_attributes_FE(df):
  for i in ['firmageobserved']:
      
      df[i+'_value'] = df[i].apply(lambda x: 0 if x < 0 else x)
      df[i+'_flag'] = df[i].apply(lambda x: 1 if x < 0 else 0)
      df.drop(i,axis = 'columns',inplace = True)

In [ ]:
def firmographic_FE_categorical(df):
  df[['firmnonprofit_f1', 'firmnonprofit_f2', 'firmnonprofit_f3',
      'firmirsretirementplan_f1',"firmirsretirementplan_f2", "firmirsretirementplan_f3",
      'firmownershiptype_f1', 'firmownershiptype_f2', 'firmownershiptype_f3',
      'firmparentcompanyind_f1', 'firmparentcompanyind_f2',  'firmparentcompanyind_f3']] = 0
  
  for i in ['firmnonprofit', 'firmirsretirementplan', 'firmownershiptype',
            'firmparentcompanyind']:
        df[i+"_f1"] = df[i].apply(lambda x: 1 if x== -1 else 0)
        df[i+"_f2"] = df[i].apply(lambda x: 1 if x == 0 else 0)
        df[i+"_f3"] = df[i].apply(lambda x: 1 if x == 1 else 0)
        df.drop(i, axis = 'columns', inplace = True)

In [ ]:
# attribute_name : firmownershiptype
#  0.0    2874
# -1.0     508

# attribute_name : firmparentcompanyind
#  1.0    2874
# -1.0     508

In [ ]:
df = pd.get_dummies(df, columns = ['firmemployeerangecount', 'firmreportedsalesrange'])
df.shape

(3382, 28)

In [ ]:
firmographic_attributes_FE(df)
firmographic_FE_categorical(df)
df.head()

,OpportunityID,WriteOff_YN,firmemployeerangecount_-1.0,firmemployeerangecount_0.0,firmemployeerangecount_10.0,firmemployeerangecount_20.0,firmemployeerangecount_30.0,firmemployeerangecount_40.0,firmemployeerangecount_50.0,firmemployeerangecount_60.0,...,firmnonprofit_f3,firmirsretirementplan_f1,firmirsretirementplan_f2,firmirsretirementplan_f3,firmownershiptype_f1,firmownershiptype_f2,firmownershiptype_f3,firmparentcompanyind_f1,firmparentcompanyind_f2,firmparentcompanyind_f3
0,0062M00000gYqZQQA0,0.0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,1
1,0062M00000gYSpGQAW,0.0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
2,0062M00000gYsQcQAK,0.0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
3,0062M00000n0BEfQAM,0.0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
4,0062M00000gYTaCQAW,1.0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,1


In [ ]:
df.shape

(3382, 37)

In [ ]:
df.columns

Index(['OpportunityID', 'WriteOff_YN', 'firmemployeerangecount_-1.0',
       'firmemployeerangecount_0.0', 'firmemployeerangecount_10.0',
       'firmemployeerangecount_20.0', 'firmemployeerangecount_30.0',
       'firmemployeerangecount_40.0', 'firmemployeerangecount_50.0',
       'firmemployeerangecount_60.0', 'firmreportedsalesrange_-1.0',
       'firmreportedsalesrange_0.0', 'firmreportedsalesrange_5.0',
       'firmreportedsalesrange_10.0', 'firmreportedsalesrange_20.0',
       'firmreportedsalesrange_30.0', 'firmreportedsalesrange_40.0',
       'firmreportedsalesrange_50.0', 'firmreportedsalesrange_60.0',
       'firmreportedsalesrange_70.0', 'firmreportedsalesrange_80.0',
       'firmreportedsalesrange_90.0', 'firmreportedsalesrange_100.0',
       'firmageobserved_value', 'firmageobserved_flag', 'firmnonprofit_f1',
       'firmnonprofit_f2', 'firmnonprofit_f3', 'firmirsretirementplan_f1',
       'firmirsretirementplan_f2', 'firmirsretirementplan_f3',
       'firmownershiptype_f1

In [ ]:
# df['firmemployeerangecount_f1'] = df['firmemployeerangecount'].apply(lambda x:0 if x <= 0 else x)
# df['firmemployeerangecount_f2'] = df['firmemployeerangecount'].apply(lambda x:1 if x > 0 and x <= 40 else 0)
# df['firmemployeerangecount_f2'] = df['firmemployeerangecount'].apply(lambda x:2 if x > 40 else 0)

In [ ]:
# df['firmreportedsalesrange_f1'] = df['firmreportedsalesrange'].apply(lambda x:0 if x <= 0 else x)
# df['firmreportedsalesrange_f2'] = df['firmreportedsalesrange'].apply(lambda x:1 if x > 0 and x <= 40 else 0)
# df['firmreportedsalesrange_f2'] = df['firmreportedsalesrange'].apply(lambda x:2 if x > 40 else 0)

In [ ]:
# firmageobserved          -0.1026
# firmemployeerangecount   -0.0811
# firmreportedsalesrange   -0.0708
# firmemployeecount        -0.0381
# firmirsretirementplan    -0.0263
# firmnonprofit            -0.0191
# firmparentcompanyind     -0.0182
# firmownershiptype        -0.0182
# firmreportedsales        -0.0142
# firmreportedearnings      0.0101

In [ ]:
round(df.corr()['WriteOff_YN'].sort_values(),4)

firmageobserved_value          -0.1032
firmreportedsalesrange_40.0    -0.0537
firmreportedsalesrange_70.0    -0.0518
firmemployeerangecount_30.0    -0.0497
firmreportedsalesrange_50.0    -0.0359
firmemployeerangecount_20.0    -0.0349
firmirsretirementplan_f3       -0.0335
firmemployeerangecount_10.0    -0.0321
firmemployeerangecount_40.0    -0.0316
firmreportedsalesrange_10.0    -0.0281
firmemployeerangecount_50.0    -0.0267
firmownershiptype_f2           -0.0182
firmparentcompanyind_f3        -0.0182
firmnonprofit_f2               -0.0161
firmirsretirementplan_f2       -0.0086
firmreportedsalesrange_30.0    -0.0074
firmnonprofit_f3               -0.0067
firmreportedsalesrange_20.0    -0.0044
firmreportedsalesrange_80.0    -0.0041
firmreportedsalesrange_100.0   -0.0016
firmreportedsalesrange_60.0     0.0016
firmemployeerangecount_0.0      0.0044
firmreportedsalesrange_0.0      0.0051
firmreportedsalesrange_5.0      0.0081
firmageobserved_flag            0.0155
firmirsretirementplan_f1 

In [ ]:
df.to_excel("/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/v11/firmographic_attributes_FE.xlsx", index = False)